导入必要的工具包

In [49]:
import pandas as pd
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.model_selection import train_test_split
from sklearn import metrics
#降维算法
from sklearn.decomposition import PCA
import time

import matplotlib.pyplot as plt
%matplotlib inline

In [50]:
#读取训练数据
train = pd.read_csv('events.csv')
#显示头5行信息
train.head()

,event_id,user_id,start_time,city,state,zip,country,lat,lng,c_1,...,c_92,c_93,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other
0,684921758,3647864012,2012-10-31T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,2,...,0,1,0,0,0,0,0,0,0,9
1,244999119,3476440521,2012-11-03T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,2,...,0,0,0,0,0,0,0,0,0,7
2,3928440935,517514445,2012-11-05T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,12
3,2582345152,781585781,2012-10-30T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,8
4,1051165850,1016098580,2012-09-27T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,9


In [51]:
#查看模型信息
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3137972 entries, 0 to 3137971
Columns: 110 entries, event_id to c_other
dtypes: float64(2), int64(103), object(5)
memory usage: 2.6+ GB


拿出所有object 类型的数据

In [52]:
trainObject = train.select_dtypes(include=['object'])

In [53]:
trainObject

,start_time,city,state,zip,country
0,2012-10-31T00:00:00.001Z,NaN,NaN,NaN,NaN
1,2012-11-03T00:00:00.001Z,NaN,NaN,NaN,NaN
2,2012-11-05T00:00:00.001Z,NaN,NaN,NaN,NaN
3,2012-10-30T00:00:00.001Z,NaN,NaN,NaN,NaN
4,2012-09-27T00:00:00.001Z,NaN,NaN,NaN,NaN
5,2012-11-16T00:00:00.001Z,NaN,NaN,NaN,NaN
6,2012-11-02T20:00:00.003Z,NaN,NaN,NaN,NaN
7,2012-10-31T00:00:00.001Z,NaN,NaN,NaN,NaN
8,2012-10-31T00:00:00.001Z,NaN,NaN,NaN,NaN
9,2012-10-18T00:00:00.001Z,NaN,NaN,NaN,NaN


找来找去 找不到资料处理 这几列   决定删除了

In [54]:
allTrain = train.drop(['start_time','city','state','zip','country','lat','lng'],axis=1)

In [55]:
#设置拿1000个数据训练
n_trains = 1000

#拿到前1000条数据
y_train = allTrain.event_id.values[:n_trains]
#从训练集中删除这一列的前1000条
X_train = allTrain.drop("event_id",axis=1).values[:n_trains]

In [56]:
allTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3137972 entries, 0 to 3137971
Columns: 103 entries, event_id to c_other
dtypes: int64(103)
memory usage: 2.4 GB


In [61]:
#设置拿1000个数据训练
n_trains = 1000
#拿到前1000条数据
y_train = allTrain.event_id.values[:n_trains]
#从训练集中删除这一列的前1000条
X_train = allTrain.drop("event_id",axis=1).values[:n_trains]

In [62]:
# 将训练集合拆分成训练集和校验集，在校验集上找到最佳的模型超参数（PCA的维数）
X_train_part, X_val, y_train_part, y_val = train_test_split(X_train,y_train, train_size = 0.8,random_state = 0)
#X_train x轴的训练集
#X_val  x轴的测试集
#y_train_part  y轴的训练集
#y_val  y轴的测试集

#X_train_pca x轴的训练数据
#y_train y轴的训练数据
#train_size = 0.8  训练集的比例
#random_state = 0  随机的次数

E:\Anaconda\envs\python3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [63]:
#拆分后的训练集和校验集的样本数目
print(X_train_part.shape)
print(X_val.shape)

(800, 102)
(200, 102)


In [64]:
# 一个参数点（聚类数据为K）的模型，在校验集上评价聚类算法性能
def K_cluster_analysis(K, X_train, y_train, X_val, y_val):
#K  分成多少类的值
#X_train_part  x轴的训练集
#y_train_part  y轴的训练集
#X_val  x轴的测试集
 #y_val  y轴的测试集
    
    #定义开始时间
    start = time.time()
    
    #输出
    print("K-means begin with clusters: {}".format(K));
    
    #K-means,在训练集上训练
    mb_kmeans = MiniBatchKMeans(n_clusters = K)
    #n_clusters = K  定义有多少类
    
    #训练模型
    mb_kmeans.fit(X_train)
    #X_train x轴的训练数据
    
    # 在训练集和测试集上测试
    #y_train_pred = mb_kmeans.fit_predict(X_train)
    y_val_pred = mb_kmeans.predict(X_val)
    #X_val x轴的测试数据
    
    #以前两维特征打印训练数据的分类结果
    #plt.scatter(X_train[:, 0], X_train[:, 1], c=y_pred)
    #plt.show()

    # K值的评估标准
    #常见的方法有轮廓系数Silhouette Coefficient和Calinski-Harabasz Index
    #这两个分数值越大则聚类效果越好
    #CH_score = metrics.calinski_harabaz_score(X_train,mb_kmeans.predict(X_train))
    CH_score = metrics.silhouette_score(X_train,mb_kmeans.predict(X_train))#内部评价
    #X_train  x轴的训练数据
    #mb_kmeans.predict(X_train)  模型预测的结果
    
    #也可以在校验集上评估K
    v_score = metrics.v_measure_score(y_val, y_val_pred)#外部评价
    #y_val  x轴的测试集
    #y_val_pred  y轴的测试集
    
    #定义结束时间
    end = time.time()
    #输出
    print("CH_score: {}, time elaps:{}".format(CH_score, int(end-start)))
    print("v_score: {}".format(v_score))
    
    return CH_score,v_score

In [65]:
# 设置超参数（聚类数目K）搜索范围
Ks = [10, 20, 30,40,50,60] #定义要分多少类
CH_scores = []  #内部评价存储列表
v_scores = []   #外部评价存储列表


for K in Ks:
    #调用方法
    ch,v = K_cluster_analysis(K, X_train_part, y_train_part, X_val, y_val)
    #K  分成多少类的值
    #X_train_part  x轴的训练集
    #y_train_part  y轴的训练集
    #X_val  x轴的测试集
    #y_val  y轴的测试集
    
    #添加到内部评价列表中
    CH_scores.append(ch)
    #添加到外部评价列表中
    v_scores.append(v)

K-means begin with clusters: 10
CH_score: 0.5537145304580585, time elaps:0
v_score: 0.596794564910414
K-means begin with clusters: 20
CH_score: 0.5700141471021556, time elaps:0
v_score: 0.7116292267173178
K-means begin with clusters: 30
CH_score: 0.5565900253233336, time elaps:0
v_score: 0.7712153192105735
K-means begin with clusters: 40
CH_score: 0.5707408539513174, time elaps:0
v_score: 0.8097981746388326
K-means begin with clusters: 50
CH_score: 0.5843962838114589, time elaps:0
v_score: 0.829226934682838
K-means begin with clusters: 60
CH_score: 0.5903576151604433, time elaps:0
v_score: 0.8482868983285813


明显分得越多分数越高

In [66]:
# 设置超参数（聚类数目K）搜索范围
Ks = [70, 80, 90,100,110,120] #定义要分多少类
CH_scores = []  #内部评价存储列表
v_scores = []   #外部评价存储列表


for K in Ks:
    #调用方法
    ch,v = K_cluster_analysis(K, X_train_part, y_train_part, X_val, y_val)
    #K  分成多少类的值
    #X_train_part  x轴的训练集
    #y_train_part  y轴的训练集
    #X_val  x轴的测试集
    #y_val  y轴的测试集
    
    #添加到内部评价列表中
    CH_scores.append(ch)
    #添加到外部评价列表中
    v_scores.append(v)

K-means begin with clusters: 70
CH_score: 0.6024311811573146, time elaps:0
v_score: 0.8642536822446226
K-means begin with clusters: 80
CH_score: 0.595187685909095, time elaps:0
v_score: 0.8783879898102758
K-means begin with clusters: 90
CH_score: 0.5949963281477919, time elaps:0
v_score: 0.8821515649538845
K-means begin with clusters: 100
CH_score: 0.5975170567704119, time elaps:0
v_score: 0.8883004198899709
K-means begin with clusters: 110
CH_score: 0.6051802958624992, time elaps:0
v_score: 0.8965214373458458
K-means begin with clusters: 120
CH_score: 0.6026567003770726, time elaps:0
v_score: 0.8966768759556525


再尝试一次

In [68]:
# 设置超参数（聚类数目K）搜索范围
Ks = [130, 140, 150,160,170,180] #定义要分多少类
CH_scores = []  #内部评价存储列表
v_scores = []   #外部评价存储列表


for K in Ks:
    #调用方法
    ch,v = K_cluster_analysis(K, X_train_part, y_train_part, X_val, y_val)
    #K  分成多少类的值
    #X_train_part  x轴的训练集
    #y_train_part  y轴的训练集
    #X_val  x轴的测试集
    #y_val  y轴的测试集
    
    #添加到内部评价列表中
    CH_scores.append(ch)
    #添加到外部评价列表中
    v_scores.append(v)

K-means begin with clusters: 130
CH_score: 0.6008241512194061, time elaps:0
v_score: 0.9082293411602252
K-means begin with clusters: 140
CH_score: 0.6003461429586057, time elaps:0
v_score: 0.9134305860043511
K-means begin with clusters: 150
CH_score: 0.6065048680740148, time elaps:0
v_score: 0.9203270426128414
K-means begin with clusters: 160
CH_score: 0.6091339452886593, time elaps:0
v_score: 0.9208262683809636
K-means begin with clusters: 170
CH_score: 0.5850990317958, time elaps:0
v_score: 0.9288485959448357
K-means begin with clusters: 180
CH_score: 0.5762343684955078, time elaps:0
v_score: 0.9269387710660978


最好的参数是 120